## Section 1: Data loading and manipulation

Work with two files that are located via dropbox:
1. `vending_machine.csv`: A table that shows the code to be entered in the vending machine keyboard to get a certain snack. It also contains the amount of products in each slot. The file can be read from this addres: https://www.dropbox.com/s/vy61ldio3t02lqy/vending_machine.csv?raw=1

2. `stock.csv`: A table with the name of each snack (without capitalisation), the category and the price. The file can be read from this addres: https://www.dropbox.com/s/qwl4i3slj4mklsj/stock.csv?raw=1

By running the following code cells, you can to read and print the two files as pandas data frames:

In [ ]:
import pandas as pd

vending_machine = pd.read_csv('https://www.dropbox.com/s/vy61ldio3t02lqy/vending_machine.csv?raw=1')
vending_machine

In [ ]:
stock = pd.read_csv('https://www.dropbox.com/s/qwl4i3slj4mklsj/stock.csv?raw=1')
stock

To use these data frames in your program, first you need to **merge** them as a single data frame which first requires item names to be the same on both dataframes.

In [ ]:
def change_name(name):
    if "irn-bru" in name:
        for word in name.split("'s"):
            name = word[0 : ]
            name = name.lstrip("' ")
            name = name[0].upper() + name[1:]
        return name
    if "'s" in name:
        for word in name.split("'s"):
            name = word[0 : ]
            name = name.lstrip("'s ").title()
        return name
    else:
        for word in name.split("s' "):
            name = word[0 : ]
            name = name.title()
        return name

stock['snack'] = stock['snack'].apply(change_name)
stock

In [ ]:
full = pd.merge(vending_machine, stock, how="left", on="snack")


indexed = full.set_index('machine_code')

indexed

### Adding Three Cans of Irn Bru

In [ ]:
indexed.loc['B4', ["snack", "amount", "category", "price"]] = ["Irn-bru", "3.0", "drinks", "3.00"]
 

indexed

## Section 2: Vending Machine Simulation Program

Once that you have generated your merged dataset, you will use the following code cell to write a small program with the following *four* options:

1. **Admin login**: This option will simulate how the vending machine operator validates their credentials. When this option is selected, the user should be prompted to input their username and password. The username should be *admin* and the password is be any number smaller than *10*. You should allow the user to try *four* times, if all of them are incorrect, then the program stops and the program cell needs to be run again.

  * Note: Once that an admin has logged in, this option has to be disabled (i.e. the user cannot go back and select this option).

2. **Add products**: This option can only be accessed once option 1 has been completed, otherwise the message *login first!* should be displayed and the program should go back to the main menu. In this option, the admin will be shown the list of snacks and categories in the vending machine. Then, the admin can input a machine code slot to add *one more product* for that certain slot. Afterwards, the program should show the new stock and go back to the main menu.
  
  * Note: Keep in mind that each space in the vending machine can hold a maximum of *eight* products (of the same snack, of course). Also, you cannot add products in the empty slots. Therefore, your program should warn the admin in case that they want to add more products of a specific snack in a slot, or if the admin wants to add products in an empty slot.

3. **Buy snacks**: This option can be accessed by "anyone", so there is no need for a validation. If this option is selected, the user will be shown the list of snacks, amounts and prices. Then, the user will be requested to select one snack based on the `machine_code`. Once the snack is selected, your program must display the price of the selected product. Then, the user will be prompted to pay. To simulate this payment, you will ask the user for an input and write any positive number (if the user inputs something invalid, ask to try again). Then, your program must check this number against the price of the product to be bought. If the input number is larger than the price, then you should return *your change is...* and the subtraction of the payment minus the price of the snack. If the number is equal to the price, then you must return *thanks for paying*. Else, you should output *you need to pay more* and allow the user to write another amount. After the "purchase", you should output to the user the number of products left for that particular snack (therefore, you need to update the data frame!). Notice that if there were zero products left for the snack selected, then you must prompt this to the user in advance before letting them buy, and ask them to select another snack.

4. **Exit**

In [ ]:
#formatting DF before running

indexed = indexed.astype({"amount": float, "price" : float})
indexed['amount'] = indexed['amount'].fillna(0)
indexed = indexed.astype({"price": float, "price" : float})
indexed['price'] = indexed['price'].fillna(0)
indexed = indexed.drop('category', 1)

In [ ]:
#function for adding snacks within overall Vending Machine programme

def add_snack():
    print(indexed)
    loop_two = True
    while loop_two == True:
        code = input("Enter Code:")
        check = full['machine_code'] == code
        if check.any():
            x = indexed.at[code, 'amount']
            snack_name = indexed.at[code, 'snack']
            if snack_name == '' or pd.isnull(snack_name): 
                print("Slot empty, please select a valid slot")
            else:
                if x >= 8:
                    print("Slot full, please select a valid slot")
                if x < 8:
                    indexed.at[code, 'amount'] = x + 1
                    print(f"Succesfully added one {indexed.at[code, 'snack']} at position {code}")
                    print(indexed)
                    loop_two = False
        else:
            print("Please select a valid code")


In [ ]:
#function for purchasing snacks contained within overall Vending MAchine programme

def buy_snack():
    loop_three = True
    print(indexed)
    while loop_three == True:
        snack = input("Please enter code of desired snack: ")
        check = full['machine_code'] == snack
        if check.any():
            x = indexed.at[snack, 'amount']
            if x == 0:
                print("Slot empty, please select a valid slot")
            if x > 0:
                price = indexed.at[snack, 'price']
                print(f"You have selected {indexed.at[snack, 'snack']}, the price is £{price:.2f}")
                money = input("Please Insert Cash: £")
                try:
                    money = float(money)
                    loop_four = True
                    while loop_four == True:
                        if money < price: 
                            print(f"The price is £{price:.2f}, you have entered £{money:.2f}. £{price - money:.2f} still to pay")
                            money = money + float(input('Please Insert Cash: £'))
                        if money >= price:
                            indexed.at[snack, 'amount'] = x - 1
                            print(indexed)
                            print(f"Here is your item, your change is £{money - price:.2f}")
                            loop_three = False
                            loop_four = False
                except ValueError:
                    print(f'INVALID INPUT, returning to purchasing screen menu. Here is your inserted cash: £{money}')
        else:
            print("Please enter a valid code")


# The Vending Machine

In [ ]:
def check_input(p):    #For checing user input is float
    try:
        p = float(p)
    except ValueError:
        return False

def password_check():  #Error messages if admin login incorrect
    if counter == 3:
        print(f'USERNAME OR PASSWORD INCORRECT FINAL ATTEMPT')
    else:
        print(f'USERNAME OR PASSWORD INCORRECT, {4 - counter} attempts remaining')

counter = 0
admin = 0
loop = True
while loop == True and counter < 4:
    print("Please select your option")
    print("Option 1: Admin Login")
    print("Option 2: Add Snacks (requires admin login)")
    print("Option 3: Buy Snacks")
    print("Option 0: Exit")
    opt = input(">>")
    
    if opt == "1" and admin == 0:
        user = input('Username:')
        password = input('Password:')
        if check_input(password) == False:
            counter = counter + 1
            password_check()
        else:
            password = float(password)
            if user == 'admin' and password < float('10'):
                admin = admin + 1
                print("ADMIN PRIVILEGES ENABLED")
                opt = None
            else:
                counter = counter + 1
                password_check()
                
    if opt == "1" and admin == 1:
        print("ALREADY LOGGED IN")
      
    if opt == "2":
        if admin == 1:
            add_snack()
        else:
            print("PLEASE LOGIN FIRST")
    
    if opt == "3":
        buy_snack()

        
    elif opt == "0":
        print("Thankyou, goodbye!")
        loop = False